In [1]:
import wget
import os
import json

In [4]:
# 定义文件路径
url_txt_path = "./download_url/url-20-25.txt"
download_path = "D:/zxr_download/Fits/20-25"

# 开始下载

In [5]:
def save_progress(progress, progress_file):
    with open(progress_file, 'w') as f:
        json.dump(progress, f)

def load_progress(progress_file):
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)
    return {'completed': [], 'err_log': {}, 'n': 0}

def download_spectra(url_txt_path, download_path, progress_file):
    with open(url_txt_path, 'r') as f:
        err_list = []
        ls = []

        # 读到链接列表ls
        f.readline()  # 跳过第一行，如果需要
        while True:
            line = f.readline()
            if line:
                ls.append(line.strip())  # 去掉换行符
            else:
                break

    # Number of all spectra
    print("Number of spectra to download:", len(ls))
    
    # 防止出问题
    progress = load_progress(progress_file)
    completed = set(progress['completed'])
    err_log = progress['err_log']
    n = progress['n']

    for j in range(0, len(ls)):
        if j in completed:
            continue
        try:
            wget.download(ls[j], download_path)
            completed.add(j)
            if j % 20 == 0:  # 每下载20个保存一次进度
                save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)
        except Exception as e:
            print(f"|| {j} err: {e}")
            err_list.append(j)
            continue

    err_log[n] = err_list
    n += 1
    save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)

    # 失败处重新下载
    while len(err_list) > 20:
        final_err_list = []
        print("\n重新下载失败光谱")
        print("Number of err spectra to download:", len(err_list))
        for j in err_list:
            try:
                wget.download(ls[j], download_path)
                completed.add(j)
                save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)
            except Exception as e:
                print(f"|| {j} err: {e}")
                final_err_list.append(j)
                continue

        err_log[n] = final_err_list
        err_list = final_err_list
        n += 1
        save_progress({'completed': list(completed), 'err_log': err_log, 'n': n}, progress_file)

    return err_log

In [ ]:
# 保存下载进度
progress_file = 'download_progress.json'

#创建下载路径的文件夹
os.makedirs(download_path, exist_ok=True)

error_log = download_spectra(url_txt_path, download_path, progress_file)

# 打印错误日志
print(error_log)

Number of spectra to download: 639027
100% [..............................................................................] 58387 / 58387|| 221 err: HTTP Error 502: Bad Gateway
 14% [..........                                                                    ]  8192 / 58348

In [ ]:
# 手动下载失败项

err_list=error_log[n-1]
final_err_list = []

print("Number of err spectra to download：", len(err_list))
for j in err_list:
    try:
        pass
        wget.download(ls[j], download_path)
    except: 
        print("|| {}  err".format(j))
        final_err_list.append(j)
        continue
        
error_log[n]=final_err_list 
n=n+1 

In [10]:
err_log

{0: [0, 1, 2, 3, 5, 8, 10, 11, 14],
 1: [2, 3, 5, 8, 10, 11, 14],
 2: [2, 3, 5, 8, 10, 11, 14],
 3: [2, 5, 8, 10, 11, 14],
 4: [2, 5, 8, 10, 11, 14]}